In [1]:
# %pip install langchain langchain_community langchain_openai llama_index llama-index-llms-langchain duckduckgo-search

In [2]:
import sys
sys.path.append('../')

from scripts.helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os

directory = "../files"

files = os.listdir(directory)

papers = [os.path.join(directory, file) for file in files]

print(papers)

['../files/Tenacious Talent - GenAI Upskilling.pdf']


In [5]:
from scripts.utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    name = Path(paper).stem.replace(' ', '_').replace('.pdf', '')
    vector_tool, summary_tool = get_doc_tools(paper, name)    
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: ../files/Tenacious Talent - GenAI Upskilling.pdf


In [6]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [7]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [8]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [9]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [10]:
tools = obj_retriever.retrieve(
    "What is Tenacious Talent - GenAI UpSkilling GEN-AI Challenge?"
)

In [11]:
tools[1].metadata

ToolMetadata(description='vector_tool_Tenacious_Talent_-_GenAI_Upskilling(query: str, page_numbers: Optional[List[str]] = None) -> str\nUse to answer questions over a given document.\n\n        Useful if you have specific questions over the document.\n        Always leave page_numbers as None UNLESS there is a specific page you want to search for.\n\n        Args:\n            query (str): the string query to be embedded.\n            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE \n                if we want to perform a vector search\n                over all pages. Otherwise, filter by the set of specified pages.\n        \n        ', name='vector_tool_Tenacious_Talent_-_GenAI_Upskilling', fn_schema=<class 'pydantic.v1.main.vector_tool_Tenacious_Talent_-_GenAI_Upskilling'>, return_direct=False)

In [12]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given documents.
Please always use the tools provided to answer a question. and give analysis based on your knowledge and the documents\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [13]:
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun

# Initialize DuckDuckGoSearch tool
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGoSearch',
    func=search.run,
    description='Use this tool to perform an internet search if document retrieval does not provide sufficient information.'
)

# Combine document tools and web search tool
# tools = [duckduckgo_tool] + all_tools

In [14]:
import os
import logging
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize logging
logging.basicConfig(level=logging.INFO)

def get_openai_client():
    """Get OpenAI client with API key."""
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("Missing OpenAI API key")
    return OpenAI(api_key=api_key)

def is_insufficient_response(response):
    """Check if the response contains phrases indicating insufficiency."""
    insufficient_phrases = [
        "is not a term or concept mentioned",
        "not found in the document",
        "not mentioned in the provided document",
        "lack of existing context"
        "does not contain"
    ]
    for phrase in insufficient_phrases:
        if phrase in response.lower():
            return True
    return False

def query_with_fallback(agent, question):
    # First, use the agent to retrieve information from documents
    response = agent.query(question)
    
    # Extract content from the response object
    content = getattr(response, 'response', None)
    
    # Debugging: print the structure of the response and content
    print(f"Response from agent: {response}")
    print(f"Extracted Content: {content}")
    
    # Check if the content is non-empty and has meaningful information
    if content and content.strip():
        # Check for specific phrases indicating insufficient response
        if is_insufficient_response(content):
            print("Detected insufficient response based on content analysis. Falling back to web search.")
            web_response = search.run(question)
            return web_response
        
        # Construct sufficiency check prompt
        sufficiency_check_prompt = f"""
        Question: {question}
        Response: {content}

        Is the above response sufficient and relevant to answer the question? Please answer with 'yes' or 'no' and provide a brief justification.
        """
        
        # Debugging: print the sufficiency check prompt
        print("Sufficiency Check Prompt:")
        print(sufficiency_check_prompt)
        
        try:
            # Perform the sufficiency check using OpenAI API
            client = get_openai_client()
            chat_completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an AI designed to assess the sufficiency and relevance of responses."},
                    {"role": "user", "content": sufficiency_check_prompt}
                ]
            )
            
            sufficiency_check = chat_completion.choices[0].message.content.strip()
            
            # Debugging: print the sufficiency check result
            print("Sufficiency Check Result:")
            print(sufficiency_check)

            # Determine if the response is sufficient based on the sufficiency check
            if 'yes' in sufficiency_check.lower():
                return content
            else:
                print("Response deemed insufficient. Falling back to web search.")
                web_response = search.run(question)
                return web_response
        except Exception as e:
            logging.error(f"Failed to get chat completion: {e}")
            return "An error occurred while trying to assess the sufficiency of the response."
    else:
        print("Document retrieval did not provide sufficient context. Falling back to web search.")
        web_response = search.run(question)
        return web_response

In [15]:
response = query_with_fallback(agent, "can you tell me about Ethiopia?")
print("\nFinal Response:", response)

Added user message to memory: can you tell me about Ethiopia?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: summary_tool_Tenacious_Talent_-_GenAI_Upskilling with args: {"input": "Ethiopia"}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Function Output ===
Team-Mate, an independent company dedicated to helping students and trainees manage their time and focus more effectively, introduces a personalized agent to enhance the training and educational experience. This system serves as a supportive digital assistant to help students manage their tasks effectively, facilitate collaboration, and access necessary resources effortlessly. The key services provided by Team-Mate include personalized interaction, adaptive learning and support, proactive planning and scheduling, blocker resolution, and enhancing collaboration. The project aims to showcase the potential of AI agents in transforming education and training environments, demonstrating how personalized learning can significantly improve outcomes and engagement.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== LLM Response ===
Ethiopia is a country in East Africa known for its rich history and diverse culture. It is home to ancient civilizations, such as the Aksumite Empire, and is one of the oldest countries in the world. Ethiopia is also famous for being the only African country that was never colonized by a European power. The country has a unique calendar system with 13 months, and it is known for its traditional cuisine, including injera (a sourdough flatbread) and various spicy stews.

Ethiopia is a land of natural beauty, with stunning landscapes that include the Simien Mountains, the Danakil Depression (one of the hottest places on Earth), and the Blue Nile Falls. The country is also known for its wildlife, including rare species like the Ethiopian wolf and the Gelada baboon.

In recent years, Ethiopia has experienced significant economic growth and development, with a focus on infrastructure projects and industrialization. The capital city, Addis Ababa, is a bustling metropolis 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Sufficiency Check Result:
Yes. 

The response provides a comprehensive overview of Ethiopia, covering its history, culture, geography, wildlife, economy, and global significance. It answers the question by offering relevant information about the country.

Final Response: Ethiopia is a country in East Africa known for its rich history and diverse culture. It is home to ancient civilizations, such as the Aksumite Empire, and is one of the oldest countries in the world. Ethiopia is also famous for being the only African country that was never colonized by a European power. The country has a unique calendar system with 13 months, and it is known for its traditional cuisine, including injera (a sourdough flatbread) and various spicy stews.

Ethiopia is a land of natural beauty, with stunning landscapes that include the Simien Mountains, the Danakil Depression (one of the hottest places on Earth), and the Blue Nile Falls. The country is also known for its wildlife, including rare species lik

In [16]:
response = query_with_fallback(agent, "What is Tenacious Talent - GenAI UpSkilling GEN-AI Challenge?")
print("\nFinal Response:", response)

Added user message to memory: What is Tenacious Talent - GenAI UpSkilling GEN-AI Challenge?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: summary_tool_Tenacious_Talent_-_GenAI_Upskilling with args: {"input": "Tenacious Talent - GenAI UpSkilling GEN-AI Challenge"}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Function Output ===
TeamMate, a fictional company, aims to assist students and trainees in managing their time and focus effectively. They introduce a personalized LLM-enhanced agent to serve as a supportive digital assistant, helping individuals manage tasks, facilitate collaboration, and access resources effortlessly. The key services provided by Team-Mate include personalized interaction, adaptive learning and support, proactive planning and scheduling, blocker resolution, and enhancing collaboration. The project utilizes technologies like LLMs and Retrieval Augmented Generation (RAG) to transform educational and training environments, enhancing learning efficiency and fostering a collaborative atmosphere. The learning outcomes focus on skills development in prompt engineering, RAG implementation, agent design, vector store implementation, data analysis with LLMs, and frontend development. The project aims to showcase the potential of AI agents in improving education and trainin

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== LLM Response ===
The Tenacious Talent - GenAI UpSkilling GEN-AI Challenge involves TeamMate, a fictional company, introducing a personalized LLM-enhanced agent to assist students and trainees in managing their time and focus effectively. The project focuses on providing personalized interaction, adaptive learning and support, proactive planning and scheduling, blocker resolution, and enhancing collaboration. Technologies like LLMs and Retrieval Augmented Generation (RAG) are utilized to transform educational and training environments, aiming to improve learning efficiency and foster a collaborative atmosphere. The project emphasizes skills development in various areas such as prompt engineering, RAG implementation, agent design, vector store implementation, data analysis with LLMs, and frontend development to showcase the potential of AI agents in enhancing education and training outcomes through personalized learning experiences.
Response from agent: The Tenacious Talent - GenAI U

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Sufficiency Check Result:
Yes, the response is sufficient and relevant to answer the question. It provides a detailed explanation of the Tenacious Talent - GenAI UpSkilling GEN-AI Challenge, outlining its objectives, methodologies, and technologies involved in enhancing education and training environments.

Final Response: The Tenacious Talent - GenAI UpSkilling GEN-AI Challenge involves TeamMate, a fictional company, introducing a personalized LLM-enhanced agent to assist students and trainees in managing their time and focus effectively. The project focuses on providing personalized interaction, adaptive learning and support, proactive planning and scheduling, blocker resolution, and enhancing collaboration. Technologies like LLMs and Retrieval Augmented Generation (RAG) are utilized to transform educational and training environments, aiming to improve learning efficiency and foster a collaborative atmosphere. The project emphasizes skills development in various areas such as prompt 

In [17]:
response = query_with_fallback(agent, "can you tell me about ALX?")
print("\nFinal Response:", response)

Added user message to memory: can you tell me about ALX?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Calling Function ===
Calling function: summary_tool_Tenacious_Talent_-_GenAI_Upskilling with args: {"input": "ALX"}


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== Function Output ===
ALX is not a term or concept mentioned in the provided context information.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=== LLM Response ===
ALX is not a term or concept mentioned in the provided document. If you have any specific information or context about ALX that you would like me to explore, please provide more details.
Response from agent: ALX is not a term or concept mentioned in the provided document. If you have any specific information or context about ALX that you would like me to explore, please provide more details.
Extracted Content: ALX is not a term or concept mentioned in the provided document. If you have any specific information or context about ALX that you would like me to explore, please provide more details.
Detected insufficient response based on content analysis. Falling back to web search.

Final Response: Tell us about the ideal ALX candidate. What qualities, experiences, mindsets, and skills do you look out for? ... You can determine if an ALX programme is the best fit for you based on your time commitment, career goals and interests. Some of our programmes require learners 